# Preprocess entire and then use existing event_samples to create Epochs anew

In [1]:
import sys
import os
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)
from src.preprocessing.classes import Subject
from src.utils.config import Config
import src.configs.config as configs
import src.preprocessing.utils as pre_utils
import src.preprocessing.utils_epo as pre_utils_epo 
from autoreject import AutoReject
import mne

# Load in subject IDs and paths from CFGLog
config = Config.from_json(configs.CFGLog)
RANDOM_STATE = config.parameters.random_seed
noisy_spans = configs.CFGLog["parameters"]["noisy_spans"]

CLBP_CP_subject_ids = config.data.chronic_low_back_pain.subject_ids.CP
CLBP_HC_subject_ids = config.data.chronic_low_back_pain.subject_ids.HC

# CLBP already processed
clbp_processed_path = config.data.chronic_low_back_pain.processed_path

# Define preprocessing parameters
# Combine all subject IDs into dict separated by group
sub_ids = {
    "chronic_low_back_pain": CLBP_CP_subject_ids + CLBP_HC_subject_ids
}

print(f"Total subjects: {len(sub_ids['chronic_low_back_pain'])}")
print(sub_ids)

event_id = configs.CFGLog['parameters']['event_id']
tmin, tmax = config.parameters.tmin, config.parameters.tmax

imported src module
imported preprocessing module
imported utils module
imported configs module
0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage
Total subjects: 43
{'chronic_low_back_pain': ['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052', '020', '021', '023', '029', '037', '041', '042', '044', '048', '049', '050', '056', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.']}


In [2]:
sub_ids = {
    "chronic_low_back_pain": ['039', '052', '056', 'C21', 'C24']
}

In [3]:
for group in sub_ids:
    for sub_id in sub_ids[group]:
        subject = Subject(sub_id, group)
        
        subject.preprocess()
        
        if not subject.file_exists("epochs_concatenated", "vhdr"):
            # Load epochs info
            subject.load_epochs_info()

            # Create epochs from events
            subject.epochs = pre_utils_epo.create_epochs_from_events(
                subject.preprocessed_raw, 
                sub_id, 
                subject.events, 
                event_id, 
                tmin, 
                tmax)
            
            # Reject and update epochs using AutoReject
            subject.reject_and_update_epochs()

            # Save
            subject.save(subject.epochs, "epochs")
            subject.save(subject.epochs.get_data(copy=False), "epochs", as_mat=True)
            
            # Concatenate epochs into VHDR for visualization
            subject.concatenate_epochs(save=True, overwrite=True)

Loaded preprocessed entire for subject 039
Loaded raw for subject 052
052
finding EOG artifacts...
Using EOG channels: EOG1, EOG2
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 10000 samples (10.000 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.9s


... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 10000 samples (10.000 s)

... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 10000 s

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.1s


... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 10000 samples (10.000 s)

Bad channels: ['Cz']
Subject: 052
Number of remaining channels:  66
Dropped channels:  []
Saved preprocessed_raw to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/052_preprocessed_raw.pkl.


/home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/src/preprocessing/classes.py:222: RuntimeWarning: Raw instance has unapplied projectors. Consider applying them before exporting with raw.apply_proj().
  data_object.export(save_file_path, overwrite=overwrite)
/home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/src/preprocessing/classes.py:222: RuntimeWarning: Encountered data in 'int' format. Converting to float32.
  data_object.export(save_file_path, overwrite=overwrite)


Saved preprocessed_raw to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/052_preprocessed_raw.vhdr.
Running autoreject on ch_type=eeg




Estimated consensus=0.60 and n_interpolate=32
Saved stimulus_labels to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/052_stimulus_labels.pkl.
Saved pain_ratings to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/052_pain_ratings.pkl.
Saved events to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/052_events.pkl.
Saved stimulus_labels to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/052_stimulus_labels.mat.
Saved pain_ratings to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/052_pain_ratings.mat.
Saved events to /home/wanglab/Documents/Geo

/tmp/ipykernel_78284/2444785389.py:25: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  subject.save(subject.epochs.get_data(), "epochs", as_mat=True)


Saved epochs to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/052_epochs.mat.


TypeError: 'NoneType' object is not iterable

Extract eyes open

In [4]:
for group in sub_ids:
    for sub_id in sub_ids[group]:
        subject = Subject(sub_id, group)
        
        subject.preprocess()
        
        if not subject.file_exists("eyes_open", "vhdr"):
            raw = subject.preprocessed_raw
            eo_annotations = annotations[2:4]  
            eo_start = eo_annotations[0]['onset']
            eo_end = eo_annotations[1]['onset']          
            subject.eyes_open = raw.copy().crop(eo_start, eo_end)
            
            subject.save(subject.eyes_open, "eyes_open")
            subject.save(subject.eyes_open, "eyes_open", as_vhdr=True, overwrite=True)
        
        # Crop noisy spans
        else:        
            subject.load_eyes_open()
            total_time_cropped = 0

            for i, crop_times in enumerate(noisy_spans[sub_id]):
                print(
                    f"Subject {sub_id}: cropping {i+1} of {len(noisy_spans[sub_id])}: {crop_times}"
                )

                # Adjust crop_times by subtracting the total time cropped
                crop_times_adjusted = [
                    crop_time - total_time_cropped for crop_time in crop_times
                ]
                crop_times = crop_times_adjusted

                # get crop times
                t1 = crop_times[0]
                t2 = crop_times[1] if len(crop_times) == 2 else None

                # if only one time is given use that as end time
                if t2 is None:
                    raw_eo = subject.eyes_open.crop(tmax=t1)
                else:
                    raw_eo = pre_utils.snip_span(subject.eyes_open, t1, t2)

            # Save cropped raw
            subject.save(raw_eo, "eyes_open", overwrite=True)
            subject.save(raw_eo, "eyes_open", as_vhdr=True, overwrite=True)
            print(f"Subject {sub_id} noisy span cropped")


Loaded preprocessed entire for subject 039
Subject 039: cropping 1 of 1: (271.5, 278.25)
Saved eyes_open to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/039_eyes_open.pkl.


/home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/src/preprocessing/classes.py:222: RuntimeWarning: Raw instance has unapplied projectors. Consider applying them before exporting with raw.apply_proj().
  data_object.export(save_file_path, overwrite=overwrite)
/home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/src/preprocessing/classes.py:222: RuntimeWarning: Encountered data in 'double' format. Converting to float32.
  data_object.export(save_file_path, overwrite=overwrite)


Saved eyes_open to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/039_eyes_open.vhdr.
Subject 039 noisy span cropped
Loaded preprocessed entire for subject 052


NameError: name 'annotations' is not defined